In [1]:
import random
import pandas as pd
import re
import math
import csv
import json
import os
import redis
from sklearn import metrics
import numpy as np

AttributeError: module 'psutil_posix' has no attribute 'getpagesize'

In [2]:
# Connect to redis

backend = redis.Redis(
    charset="utf-8",
    decode_responses=True,
    password=open("config/secrets/db-password.txt").read().strip(),
    port=6666,
)
key_values = {key: json.loads(backend.get(key)) for key in backend.keys()}

# OUTPUT_DIR = "data/db-dump"
OUTPUT_DIR = ""

In [3]:
# Just for testing purposes!

# user = 'rcdv-ann-ig:user:0605_1403'
# tmp = json.loads(backend[user])

In [21]:
# Check how many links are visited

file_name = "/data/webapps/urls.csv"
url_df = pd.read_csv(file_name, header=None)
print(len(url_df[url_df[1] == 1]), len(url_df), len(url_df)-len(url_df[url_df[1] == 1]))

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


(7, 157, 150)

In [5]:
# Retrieve user data from database and save as jsonl

# Filter user ids during debugging
user_ids = [k for k in key_values if ":user:" in k and "debug" not in k]
user_data = []

for user in user_ids:
    tmp = json.loads(backend[user])

    experiment_id = user.split(":")[0]
    user_id = user.split(":")[2]
    if len(tmp['split_id']) > 1: split_id = int(tmp['split_id'][0])
    else: split_id = int(tmp['split_id'])
    
    if 'consent-check' in tmp: consent_check = tmp['consent-check']
    else: consent_check = False

    if 'attention-check' in tmp: attention_check = tmp['attention-check']
    else: attention_check = False
    if 'attention-state' in tmp: attention_state = tmp['attention-state']
    else: attention_state = False

    if 'user-task-instances' in tmp: user_task_instances = tmp['user-task-instances']
    else: user_task_instances = False

    if 'task-state' in tmp: task_state = tmp['task-state']
    else: task_state = False

    if 'survey-check' in tmp: survey_check = tmp['survey-check']
    else: survey_check = False
    if 'survey-state' in tmp: survey_state = tmp['survey-state']
    else: survey_state = False

    user_data.append(
        {
            "experiment_id": experiment_id,
            "user_id": user_id,
            "split_id": split_id,
            "consent_check": consent_check,
            "attention_check": attention_check,
            "attention_state": attention_state,
            "user_task_instances": user_task_instances,
            "task_state": task_state,
            "survey_check": survey_check,
            "survey_state": survey_state,
        }
    )

user_df = pd.DataFrame(user_data)
user_df.to_json(os.path.join(OUTPUT_DIR, "/data/viv/user-viv.jsonl"), lines=True, orient="records")

In [6]:
# Filter old data and include only experiment data

user_data_path = "/data/viv/user-viv.jsonl"
df = pd.read_json(user_data_path, lines=True)

discarded_user_ids = [
 '62699ca4c48ff1d410fcd8f7',
 '62ba10d2cd9577af0d8744bb',
 '6282575ebe32fdc6268babc8',
 '643d681fb11898ee6b771ccf',
 '5b6515e83662a8000158b2aa',
 '63d7d95057d77b4c34b3afe8',
 '5c82c9a4d8302a00010639b8',
 '5f77c6a1a110ce346dc016ec',
 '5b5a822e1ad8270001c4f28d',
 '5e3c79de24984505f37d3903',
 '629e4d79be0e1ecd4ce30ab4',
 '5ee2e7bf21501a83d55642cd',
 '5c5a19299404f10001ff8cbf',
 '63d7e54a90879dee84dffb32', 
 '6455b4121cd3166666ae1fdd',
 '61292a3404d387db5b0b2a32',
 ]
# discarded_user_ids = []

# This line excludes rows with the following criteria:
# 1. user id that starts with 0 (we want to ignore user ids used during debugging)
# 2. attention_state, task state, survey state are false (we only want complete data)
# 3. remove user ids
filtered_df = df[(~df['user_id'].str.startswith('0')) & \
                 ~(df['attention_state'] == False) & \
                 ~(df['task_state'] == False) & \
                 ~(df['survey_state'] == False) & \
                 ~(df['user_id'].isin(discarded_user_ids))]
# filtered_df = df[(~df['user_id'].str.startswith('0'))]

print(f"len of filtered_df: {len(filtered_df)}")


263

In [7]:
# This line counts the number of participants each experiment_id has
filtered_df.groupby(['experiment_id', 'split_id']).size().reset_index(name='count')

,experiment_id,split_id,count
0,german-ann-ig,3,24
1,german-ann-lime,1,23
2,german-ann-lime,2,24
3,german-ann-lime,3,22
4,german-ann-sg,3,20
5,german-ann-shap,3,21
6,rcdv-ann-ig,3,16
7,rcdv-ann-lime,1,25
8,rcdv-ann-lime,2,26
9,rcdv-ann-lime,3,14


In [8]:
def create_senstitive_attribute_dict(key):
    tmp = json.loads(backend[key])
    assert len(tmp) == 200
    dataset = key.split(":")[0].split("-")[0]
    
    d = {}
    for i in tmp:
        sensitive_attribute = ''
        # gender for German Credit and race for rcdv
        if dataset == 'rcdv':
            sensitive_attribute = i['features']['white']
        if dataset == 'german':
            if i['features']['personal-status-sex_1'] == 1 or i['features']['personal-status-sex_3'] == 1:
                sensitive_attribute = 1 # male
            if i['features']['personal-status-sex_2'] == 1 or i['features']['personal-status-sex_5'] == 1:
                sensitive_attribute = 0 # female
        instance_id = i['instance-idx']
        d[instance_id] = sensitive_attribute
    return d


# Create task data senstive attributes dict
# to retrieve sensitive attributes more easily

task_data = [k for k in key_values if "task-data" in k and "debug" not in k]
task_data_dict = {}
for key in task_data:
    tmp_d = create_senstitive_attribute_dict(key)
    experiment_id = key.split(":")[0]
    task_data_dict[experiment_id] = tmp_d

In [9]:
filtered_df.head(5)

,experiment_id,user_id,split_id,consent_check,attention_check,attention_state,user_task_instances,task_state,survey_check,survey_state
0,rcdv-ann-lime,62b0ff84054c6ca32f481c65,3,True,True,"{'attention-start-time': 1685998608372, 'label...","[{'instance-idx': 123, 'features': {'white': 0...","{'task-start-time': 1685998616865, 'task-progr...",True,"{'survey-start-time': 1685999119572, 'labels':..."
2,german-ann-shap,63483c4522d65d46028da7a7,3,True,True,"{'attention-start-time': 1685909645393, 'label...","[{'instance-idx': 177, 'features': {'duration'...","{'task-start-time': 1685909650827, 'task-progr...",True,"{'survey-start-time': 1685909863443, 'labels':..."
11,german-ann-shap,5bebd4111296920001d55c5a,3,True,True,"{'attention-start-time': 1685959968482, 'label...","[{'instance-idx': 78, 'features': {'duration':...","{'task-start-time': 1685959976094, 'task-progr...",True,"{'survey-start-time': 1685960588231, 'labels':..."
12,german-ann-lime,6453a33ac9fac0b2749cb4e8,1,True,True,"{'attention-start-time': 1685998330234, 'label...","[{'instance-idx': 190, 'features': {'duration'...","{'task-start-time': 1685998338128, 'task-progr...",True,"{'survey-start-time': 1685999095398, 'labels':..."
13,german-ann-shap,60be1e056f39c387c4f02308,3,True,True,"{'attention-start-time': 1685960189137, 'label...","[{'instance-idx': 9, 'features': {'duration': ...","{'task-start-time': 1685960201344, 'task-progr...",True,"{'survey-start-time': 1685960688690, 'labels':..."


In [10]:
# Save each user label as a row in a dataframe

user_ids = set()

task_data = []
for idx, row in filtered_df.iterrows():
    task_labels = row['task_state']['labels']
    try:
        assert len(task_labels) == 20
    except:
        print(f"user id: {row['user_id']}")

    # Get time label on last task label
    task_keys = list(task_labels.keys())
    last_task_key = task_keys[len(task_keys)-1]
    last_task_end_time = task_labels[last_task_key]['label-time']

    # Get time label on last attention label
    attention_labels = row['attention_state']['labels']
    attention_qns_keys = list(attention_labels.keys())
    last_attention_qn_key = attention_qns_keys[len(attention_qns_keys)-1]
    attention_end_time = attention_labels[last_attention_qn_key]['label-time']

    # Find total time taken and average time taken in seconds
    # Divide by 100 since original timestamps are in milliseconds
    total_time_taken_for_task = (last_task_end_time - attention_end_time) / 100 / 60
    average_time_per_prediction = total_time_taken_for_task / len(task_labels)

    count = 0
    for index, task_label in task_labels.items():
        # Use only the first 20 labels
        if count > 20:
            break

        if 'instance' not in task_label:
            break

        instance_id = int(task_label['instance'])
        experiment_id = row['experiment_id']

        user_label = 0
        try:
            if task_label['label'] == "will_recidivate" or task_label['label'] == "good":
                user_label = 1
        except:
            user_ids.add(row['user_id'])
            break
        actual_label = int(task_label['actual_label'])
        prediction = int(task_label['prediction'])

        task_data.append(
            {
                "experiment_id": experiment_id,
                "user_id": row['user_id'],
                "split_id": row['split_id'],
                "instance_order": index,
                "instance_id": instance_id,
                "user_label": user_label,
                "actual_label": actual_label,
                "prediction": prediction,
                "sensitive_attribute": task_data_dict[experiment_id][instance_id],
                "average_time": average_time_per_prediction
            }
        )
        count += 1

task_df = pd.DataFrame(task_data)
task_df.to_json(os.path.join(OUTPUT_DIR, "/data/viv/task-viv.jsonl"), lines=True, orient="records")

print(f"user ids that need to be included in discarded: {user_ids}")


user id: 5b4d39f2d4c93b00019c8217
user id: 6469da01789c06be633d913f


set()

In [ ]:
# list(filtered_df[filtered_df['user_id'] == '61292a3404d387db5b0b2a32']['survey_state'])

# filtered_df[filtered_df['user_id'] == '61292a3404d387db5b0b2a32']

In [11]:
task_data_path = "/data/viv/task-viv.jsonl"
task_df = pd.read_json(task_data_path, lines=True)
# df.groupby(['experiment_id', 'split_id']).size().reset_index(name='count')
task_df.head(5)

,experiment_id,user_id,split_id,instance_order,instance_id,user_label,actual_label,prediction,sensitive_attribute,average_time
0,rcdv-ann-lime,62b0ff84054c6ca32f481c65,3,0,90,1,0,0,1,4.196308
1,rcdv-ann-lime,62b0ff84054c6ca32f481c65,3,1,22,0,0,0,0,4.196308
2,rcdv-ann-lime,62b0ff84054c6ca32f481c65,3,2,154,1,0,1,1,4.196308
3,rcdv-ann-lime,62b0ff84054c6ca32f481c65,3,3,39,1,0,1,1,4.196308
4,rcdv-ann-lime,62b0ff84054c6ca32f481c65,3,4,126,1,1,1,1,4.196308


In [ ]:
# user_ids = list(task_df['user_id'].unique())
# for user_id in user_ids:
#     time_taken = task_df[task_df['user_id'] == user_id]['average_time'].unique()
#     print(f"user id {user_id} took {time_taken} minutes")

In [12]:
# Analyze task data with evaluation metrics

def calculate_tpr_fpr(y_true, y_pred):
    cm = metrics.confusion_matrix(y_true, y_pred)
    if cm.shape == (1, 1):
        # Only one class present in y_true
        tn, fp, fn, tp = 0, 0, 0, cm[0, 0]
    elif cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
    else:
        raise ValueError("Unexpected confusion matrix shape: {}".format(cm.shape))
    
    # Avoid division by zero when calculating TPR and FPR
    tpr = tp / (tp + fn) if tp + fn > 0 else 0
    fpr = fp / (fp + tn) if fp + tn > 0 else 0
    return round(tpr, 3), round(fpr, 3)


def absolute_equality_of_opportunity_difference(y_true, y_pred, sensitive_attr):
    group_0_indices = (sensitive_attr == 0)
    group_1_indices = (sensitive_attr == 1)
    
    tpr_0, _ = calculate_tpr_fpr(y_true[group_0_indices], y_pred[group_0_indices])
    tpr_1, _ = calculate_tpr_fpr(y_true[group_1_indices], y_pred[group_1_indices])
    
    return np.abs(tpr_0 - tpr_1)


def absolute_equalized_odds_difference(y_true, y_pred, sensitive_attr):
    group_0_indices = (sensitive_attr == 0)
    group_1_indices = (sensitive_attr == 1)
    
    tpr_0, fpr_0 = calculate_tpr_fpr(y_true[group_0_indices], y_pred[group_0_indices])
    tpr_1, fpr_1 = calculate_tpr_fpr(y_true[group_1_indices], y_pred[group_1_indices])
    
    return (np.abs(tpr_0 - tpr_1) + np.abs(fpr_0 - fpr_1)) / 2


conditions = task_df.apply(lambda row: f"{row['experiment_id']}_{row['split_id']}", axis=1).unique()
with open("/data/viv/objective.txt", "w") as file:
    file.write(f"total conditions: {len(conditions)}\n\n")

    for condition in conditions:
        experiment_id = condition.split('_')[0]
        split_id = condition.split('_')[1]
        tmp_df = task_df[(task_df['experiment_id'] == experiment_id) & (task_df['split_id'] == int(split_id))]

        time_df = tmp_df.groupby(['experiment_id', 'split_id','user_id'])['average_time'].mean().reset_index(name='average_time_per_prediction')
        time_taken = np.mean(time_df['average_time_per_prediction'])

        over_reliance = round(len(tmp_df[(tmp_df['user_label'] == tmp_df['prediction']) & (tmp_df['actual_label'] != tmp_df['prediction'])]) / len(tmp_df), 3)
        under_reliance = round(len(tmp_df[(tmp_df['user_label'] != tmp_df['prediction']) & (tmp_df['actual_label'] == tmp_df['prediction'])]) / len(tmp_df), 3)

        preds = tmp_df['prediction']
        labels = tmp_df['actual_label']
        user_labels = tmp_df['user_label']
        sensitive_attr = tmp_df['sensitive_attribute']
        assert len(preds) == len(labels) == len(user_labels) == len(tmp_df)
        
        accuracy = round(metrics.accuracy_score(labels, preds), 3)
        precision = round(metrics.precision_score(labels, preds), 3)
        recall = round(metrics.recall_score(labels, preds), 3)
        f1 = round(metrics.f1_score(labels, preds), 3)
        tn, fp, fn, tp = metrics.confusion_matrix(labels, preds).ravel()

        tpr, fpr = calculate_tpr_fpr(labels, preds)
        aeood = round(absolute_equality_of_opportunity_difference(labels, preds, sensitive_attr), 3)
        aeod = round(absolute_equalized_odds_difference(labels, preds, sensitive_attr), 3)

        # print(f"===== {experiment_id}:{split_id} ======")
        # print(f"# of participants: {len(preds) / 20} (we want it to be around 30)")
        # print(f"# of preds/labels: {len(preds)} (we want it to be around 600)")
        # print(f"accuracy: {accuracy}")
        # print(f"precision: {precision}")
        # print(f"recall: {recall}")
        # print(f"f1: {f1}")
        # print(f"over reliance: {over_reliance}")
        # print(f"under reliance: {under_reliance}")
        # print(f"tpr: {tpr}")
        # print(f"fpr: {fpr}")
        # print(f"absolute equality of opportunity difference: {aeood}")
        # print(f"absolute equalized odds difference: {aeod}")
        # print(f"true negatives: {tn}")
        # print(f"false positives: {fp}")
        # print(f"false negatives: {fn}")
        # print(f"true positives: {tp}\n")

        file.write(f"===== {experiment_id}:{split_id} ======\n")
        file.write(f"# of participants: {len(preds) / 20} (we want it to be around 30)\n")
        file.write(f"# of preds/labels: {len(preds)} (we want it to be around 600)\n")
        file.write(f"accuracy: {accuracy}\n")
        file.write(f"precision: {precision}\n")
        file.write(f"recall: {recall}\n")
        file.write(f"f1: {f1}\n")
        file.write(f"over reliance: {over_reliance}\n")
        file.write(f"under reliance: {under_reliance}\n")
        file.write(f"tpr: {tpr}\n")
        file.write(f"fpr: {fpr}\n")
        file.write(f"absolute equality of opportunity difference: {aeood}\n")
        file.write(f"absolute equalized odds difference: {aeod}\n")
        file.write(f"true negatives: {tn}\n")
        file.write(f"false positives: {fp}\n")
        file.write(f"false negatives: {fn}\n")
        file.write(f"true positives: {tp}\n")
        file.write(f"time taken per prediction in minutes: {time_taken}\n")
        file.write("\n")

In [13]:
# Save each survey label as a row in a dataframe

likert_score_conversion = {
    "Strongly Disagree": 1,
    "Disagree": 2,
    "Neutral": 3,
    "Agree": 4,
    "Strongly Agree" : 5,
}

survey_data = []
for idx, row in filtered_df.iterrows():
    survey_labels = row['survey_state']['labels']

    for index, survey_label in survey_labels.items():
        instance_id = int(task_label['instance'])
        experiment_id = row['experiment_id']
        survey_label = survey_label['label']

        survey_data.append(
            {
                "experiment_id": experiment_id,
                "user_id": row['user_id'],
                "split_id": row['split_id'],
                "question_number": index,
                "survey_label": survey_label,
                "survey_label_score": likert_score_conversion.get(survey_label, False),
            }
        )

survey_df = pd.DataFrame(survey_data)
survey_df.to_json(os.path.join(OUTPUT_DIR, "/data/viv/survey-viv.jsonl"), lines=True, orient="records")

In [14]:
survey_data_path = "/data/viv/survey-viv.jsonl"
survey_df = pd.read_json(survey_data_path, lines=True)
# df.groupby(['experiment_id', 'split_id', 'user_id']).size().reset_index(name='count')
survey_df.head(5)

,experiment_id,user_id,split_id,question_number,survey_label,survey_label_score
0,rcdv-ann-lime,62b0ff84054c6ca32f481c65,3,0,That was very well put together. Very interest...,0
1,rcdv-ann-lime,62b0ff84054c6ca32f481c65,3,1,Agree,4
2,rcdv-ann-lime,62b0ff84054c6ca32f481c65,3,2,Neutral,3
3,rcdv-ann-lime,62b0ff84054c6ca32f481c65,3,3,Agree,4
4,rcdv-ann-lime,62b0ff84054c6ca32f481c65,3,4,Agree,4


In [15]:
# Analyze survey data

conditions = survey_df.apply(lambda row: f"{row['experiment_id']}_{row['split_id']}", axis=1).unique()
with open("/data/viv/subjective.txt", "w") as file:
    file.write(f"total conditions: {len(conditions)}\n\n")

    for condition in conditions:
        experiment_id = condition.split('_')[0]
        split_id = condition.split('_')[1]
        condition_df = survey_df[(survey_df['experiment_id'] == experiment_id) & (survey_df['split_id'] == int(split_id))]
        file.write(f"===== {experiment_id}:{split_id} ======\n")

        survey_qns = [i for i in range(1, 17)]
        if split_id == '1': 
            survey_qns = [i for i in range(1, 17) if i not in range(2, 17)]
        if split_id == '2': 
            survey_qns = [i for i in range(1, 17) if i not in [4, 10, 11, 12, 13, 14]]

        for qn in survey_qns:
            tmp_df = condition_df[(condition_df['question_number'] == qn)]
            mean = np.mean(tmp_df['survey_label_score'])
            std = np.std(tmp_df['survey_label_score'])
            # print(f"survey question {qn}")
            # print(f"mean: {round(mean, 3)}, std: {round(std, 3)}\n")

            file.write(f"{qn}. mean: {round(mean, 3)}, std: {round(std, 3)}\n")
        file.write("\n")

In [ ]:
# file_path = '/home/vivlai/openhxai/config/rcdv/codebook.csv'
# df = pd.read_csv(file_path)

# for idx, row in df.iterrows():
#     note = str(row['note'])
#     if note.strip() != '' and note.strip() != 'nan':
#         tmp = row['variable full name']
#         variable = re.sub(r'\(.*\)', '', tmp).strip()

#         print(f"<p>{variable} - {note}</p>\n")

In [ ]:
# file_path = '/home/vivlai/openhxai/config/german/codebook.csv'
# df = pd.read_csv(file_path)

# for idx, row in df.iterrows():
#     note = str(row['note'])
#     if note.strip() != '' and note.strip() != 'nan':
#         tmp = row['variable full name']
#         variable = re.sub(r'\(.*\)', '', tmp).strip()

#         print(f"<p>{variable} - {note}</p>\n")

In [16]:
# This line counts the number of participants each experiment_id has
count_df = filtered_df.groupby(['experiment_id', 'split_id']).size().reset_index(name='count')
count_df

,experiment_id,split_id,count
0,german-ann-ig,3,24
1,german-ann-lime,1,23
2,german-ann-lime,2,24
3,german-ann-lime,3,22
4,german-ann-sg,3,20
5,german-ann-shap,3,21
6,rcdv-ann-ig,3,16
7,rcdv-ann-lime,1,25
8,rcdv-ann-lime,2,26
9,rcdv-ann-lime,3,14


In [17]:
# Find out how many more splits are needed to generate in urls.csv

ideal_count_of_participants = 30
leeway = 5
total_remaining = 0

count_dict = {}
for idx, row in count_df.iterrows():
    condition = f"{row['experiment_id']}_{row['split_id']}"
    count = row['count']
    remaining = ideal_count_of_participants - count
    total_remaining += remaining
    count_dict[condition] = remaining + leeway

# print(count_dict)
print(f"remaining HITs to run: {total_remaining}")


remaining HITs to run: 97


In [18]:
condition_url_dict = {
 'rcdv-ann-lime_1': "https://openhexai1.machineintheloop.com/?userId=_&splitId=1",
 'rcdv-ann-lime_2': "https://openhexai1.machineintheloop.com/?userId=_&splitId=2",
 'rcdv-ann-lime_3': "https://openhexai1.machineintheloop.com/?userId=_&splitId=3",
 'rcdv-ann-shap_3': "https://openhexai1b.machineintheloop.com/?userId=_&splitId=3",
 'rcdv-ann-ig_3': "https://openhexai1c.machineintheloop.com/?userId=_&splitId=3",
 'rcdv-ann-sg_3': "https://openhexai1d.machineintheloop.com/?userId=_&splitId=3",
 'german-ann-lime_1': "https://openhexai2.machineintheloop.com/?userId=_&splitId=1",
 'german-ann-lime_2': "https://openhexai2.machineintheloop.com/?userId=_&splitId=2",
 'german-ann-lime_3': "https://openhexai2.machineintheloop.com/?userId=_&splitId=3",
 'german-ann-shap_3': "https://openhexai2b.machineintheloop.com/?userId=_&splitId=3",
 'german-ann-ig_3': "https://openhexai2c.machineintheloop.com/?userId=_&splitId=3",
 'german-ann-sg_3': "https://openhexai2d.machineintheloop.com/?userId=_&splitId=3",
}

all_urls = []
for condition, count in count_dict.items():
    url = condition_url_dict[condition]
    for i in range(count):
        all_urls.append(f"{url},0")

assert len(all_urls) == sum(list(count_dict.values()))

random.shuffle(all_urls)

file_name = "/data/webapps/urls.csv"
with open(file_name, 'w') as file:
    for url in all_urls:
        file.write(url + '\n')

In [19]:
file_name = "/data/webapps/urls.csv"
url_df = pd.read_csv(file_name, header=None)
print(len(url_df[url_df[1] == 1]), len(url_df), len(url_df)-len(url_df[url_df[1] == 1]))

(0, 157, 157)